In [1]:
import os
import sys
import json
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

sys.path.append('../src')

from utils import load_openai

OPENAI_API_KEY = load_openai()
FOLDER_PATH = "C:\\Users\\Stoja\\GitHub\\kp_semantic_search\\ads_json"
VECTOR_DB_PATH = "C:\\Users\\Stoja\\GitHub\\kp_semantic_search\\vector_db"
CURRENCY_CONVERTER = 117

In [2]:
#func for reformating ad ID
def rf_ad_id(data):
    try:
        return data['ad']['ad_id'].split('#')[1]
    except:
        return 'Nepoznato'
#func for reformating membership
def rf_membership(data):
    try:
        return data['user']['membership'].split(' od ')[1]
    except:
        return 'Nepoznato'
    
#func for reformating price
def rf_price(data):
    try:
        price = data['ad']['price'].replace('.', '')
    except:
        price = data['ad']['price']
    try:
        value = eval(price.split(': ')[1].split(' ')[0])
        currency = rf_currency(price)
        return value, currency
    except:
        return 'Kontakt', ''
        
        # Cena: 8.900 din
        
#func for reformating currency
def rf_currency(price):
    try:
        currency =  price.split(': ')[1].split(' ')[1]
        if currency == 'din':
            return 'din'
        else:
            return 'eur'
    except:
        return ''
    
def metadata_price(data):
    value, currency = rf_price(data)
    if (currency == 'din') or (currency == ''):
        return value
    return value * CURRENCY_CONVERTER


#func for reformating condition
def rf_condition(data):
    if data['ad']['condition'] == '':
        return 'Nepoznato'
    return data['ad']['condition']

#func for reformating description text
def rf_desc(data):
    return data['ad']['description'].replace("\n", " ")

#func for creating text for embedding
def create_embed_text(data):
    price, currency = rf_price(data)
    embedding_text = f"""
                Naziv oglasa je {data['ad']['ad_name']}.
                Oglaseni proizvod je iz kategorije {data['ad']['category']}, iz podkategorije {data['ad']['sub_cat']}.
                Cena oglasenog proizvoda je {price} {currency}.
                Stanje oglasenog proizvoda je {rf_condition(data)}.
                Dodatne informacije o proizvodu su: {rf_desc(data)}.
                Korisnik koji je oglasio proizvod je {data['user']['name']}, clan je od {rf_membership(data)} iz mesta {data['user']['place']}.
                Korisnik ima {data['user']['good_reviews']} pozitivnih ocena i {data['user']['bad_reviews']} negativnih ocena.
            """
    return embedding_text

#func for creating metadata
def create_metadata(data):
    metadata = {
        'ad_url': data['ad']['ad_url'],
        'ad_id': rf_ad_id(data),
        'price': metadata_price(data),
        'category': data['ad']['category'],
        'sub_category': data['ad']['sub_cat'],
        'positive_reviews': data['user']['good_reviews'],
        'negative_reviews': data['user']['bad_reviews'],
        'place': data['user']['place']
    }
    return metadata


In [3]:
def load_jsons(folder_path):
    json_objects = []
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    jsonl_files = [f for f in files if f.endswith('.jsonl')]

    for jsonl_file in jsonl_files:
        file_path = os.path.join(folder_path, jsonl_file)
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                    json_objects.append(json.loads(line))
    return json_objects



In [4]:
json_objects = load_jsons(FOLDER_PATH)

In [5]:
ad_contents = []
ad_metadatas = []

for jo in json_objects:
    content = create_embed_text(jo)
    metadata = create_metadata(jo)
    ad_contents.append(content)
    ad_metadatas.append(metadata)

type(ad_metadatas[5]['price'])

int

In [6]:
chroma_client = chromadb.PersistentClient(path=VECTOR_DB_PATH)
embedding_function = OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY)

# chroma_client.delete_collection(name="kp_ads")

kp_ads_collections = chroma_client.get_or_create_collection(
    name="kp_ads",
    embedding_function=embedding_function,
    metadata={"hnsw:space": "cosine"},
)

In [ ]:
for content, metadata in zip(ad_contents, ad_metadatas):
    kp_ads_collections.add(
        ids = metadata['ad_id'],
        documents = content,
        metadatas = {
            'ad_url': metadata['ad_url'],
            'ad_id': metadata['ad_id'],
            'price': metadata['price'],
            'category': metadata['category'],
            'sub_category': metadata['sub_category'],
            'positive_reviews': metadata['positive_reviews'],
            'negative_reviews': metadata['negative_reviews'],
            'place': metadata['place']}

    )

In [10]:
len(kp_ads_collections.get()['documents'])

172

In [8]:
import openai

In [9]:
def completion(query):

  prompt = f"""
        Ispod se nalazi korisnikov Upit. Tvoja uloga je da izvuces informacije
        iz oglasa koje ce se koristiti u upitu ka bazi. Korisnik moze traziti da cena
        bude manja od 1000 eura, ili da bude veca od 5000 dinara. Ispod su informacije koje treba da izvuces: 
                - Cena
                - Namera, odnosno da li korisnik zeli da pronade oglase koji su manji od Cene, ili
                  visi od Cene. Ukoliko korisnik kaze 'jeftinije od' ili 'manje od', onda je 
                  Namera='lte'. Ukoliko korisnik kaze 'skuplje od' ili 'vece od' Namera='gte'
                - Valuta, korisnik moze uneti valutu kao: 'eur' ili 'evra' ili 'eura' ili 'e' a ti u odgovoru vracas 'eur', za sve ostale vracas 'din'
                - Ostali deo teksta treba da ostavis u polje Upit
                - Ukoliko u upitu nema vrednosti koje bi odgovarale ceni, valuti ili nameri onda su one None
        Nakon sto izvuces ove informacije iz Upita, treba da napravis listu sa tim informacijama. 
        Ispod su nekoliko primera sa Upitom korisnika i Odgovorom koji treba da das. Ne treba da dajes objasnjenje vec samo jednu listu kao odgovor:        

        Upit: "Pronadji parfeme iz Beograda jeftinije od 100 evra"
        Odgovor: [100, 'eur', 'lte', 'Pronadji parfeme iz Beograda']
              
        Upit: "Pronadji sve patike skuplje od 5000 dinara"
        Odgovor: [5000, 'din', 'gte', 'Pronadji sve patike']
        
        Upit: "Pronadji parfeme iz Beograda"
        Odgovor: [None, None, None, 'Pronadji parfeme iz Beograda']

        Upit: 
        {query}

        Odgovor:

        """

  response = openai.Completion.create(
    engine="gpt-3.5-turbo-instruct",  
    prompt=prompt,
    max_tokens=100,
    temperature = 0.0
  )
  generated_text = response['choices'][0]['text']
  return generated_text


In [15]:
generated_text = completion('pronadji sve parfeme jeftinije od 100 evra') #insert query text

res = eval(generated_text)

res


[100, 'eur', 'lte', 'pronadji sve parfeme']

In [16]:
def retrieve_ads(res):
    price = res[0]
    currency = res[1]
    rule = res[2]
    query = res[3]
    if (price != None) and (currency != None) and (rule != None):
        if currency == 'din':
            result = kp_ads_collections.query(
                query_texts=query,
                n_results=5,
                include=["metadatas"],
                where={
                    "price": {
                        "$"+rule: price
                    }
                }
            )
            return result
        
        result = kp_ads_collections.query(
            query_texts=query,
            n_results=5,
            include=["metadatas"],
            where={
                "price": {
                    "$"+rule: price * CURRENCY_CONVERTER
                }
            }
        )
        return result
        
    result = kp_ads_collections.query(
        query_texts=query,
        n_results=5,
        include=["metadatas"]
    )
    return result

In [17]:
response = retrieve_ads(res)

In [18]:
print(res)
response

[100, 'eur', 'lte', 'pronadji sve parfeme']


{'ids': [['85355514', '79826117', '56886134', '34603916', '160521385']],
 'distances': None,
 'metadatas': [[{'ad_id': '85355514',
    'ad_url': 'https://www.kupujemprodajem.com/nega-lica-tela-i-ulepsavanje/parfemi-muski/viktor-rolf-spicebomb-extreme-edp-90ml/oglas/85355514',
    'category': 'Nega lica, tela i ulepšavanje',
    'negative_reviews': '0',
    'place': 'Beograd',
    'positive_reviews': '13.869',
    'price': 10764,
    'sub_category': 'Parfemi | Muški'},
   {'ad_id': '79826117',
    'ad_url': 'https://www.kupujemprodajem.com/nega-lica-tela-i-ulepsavanje/parfemi-muski/armaf-club-de-nuit-intense-man-edt105ml/oglas/79826117',
    'category': 'Nega lica, tela i ulepšavanje',
    'negative_reviews': '0',
    'place': 'Beograd',
    'positive_reviews': '13.869',
    'price': 4680,
    'sub_category': 'Parfemi | Muški'},
   {'ad_id': '56886134',
    'ad_url': 'https://www.kupujemprodajem.com/nega-lica-tela-i-ulepsavanje/parfemi-muski/mancera-cedrat-boise-edp120ml/oglas/56886134'